In [1]:
import torch
import numpy as np
import pdb
from sklearn.metrics import roc_auc_score
seed = 2298839
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
sigmoid = lambda x: 1/(1+np.exp(-x))
mse = lambda x,y: np.mean((x-y)**2)
acc = lambda x,y: np.mean(x == y)

In [6]:
import os
from dataset import load_data, rating_mat_to_sample
dataset_name = "yahoo"

if dataset_name == "coat":
    data_set_dir = os.path.join("../data", dataset_name)
    train_file = os.path.join(data_set_dir, "train.ascii")
    test_file = os.path.join(data_set_dir, "test.ascii")
    with open(train_file, "r") as f:
        train_mat = []
        for line in f.readlines():
            train_mat.append(line.split())
        train_mat = np.array(train_mat).astype(int)
    with open(test_file, "r") as f:
        test_mat = []
        for line in f.readlines():
            test_mat.append(line.split())
        test_mat = np.array(test_mat).astype(int)
    x_train, y_train = rating_mat_to_sample(train_mat)
    x_test, y_test = rating_mat_to_sample(test_mat)
    num_user = train_mat.shape[0]
    num_item = train_mat.shape[1]
else:
    data_set_dir = os.path.join("../data", dataset_name)
    train_file = os.path.join(data_set_dir, "train.txt")
    test_file = os.path.join(data_set_dir, "test.txt")
    train_mat = []
    # <user_id> <song id> <rating>
    with open(train_file, "r") as f:
        for line in f:
            train_mat.append(line.strip().split())
    train_mat = np.array(train_mat).astype(int)

    test_mat = []
    # <user_id> <song id> <rating>
    with open(test_file, "r") as f:
        for line in f:
            test_mat.append(line.strip().split())
    test_mat = np.array(test_mat).astype(int)
    print("===>Load from {} data set<===".format(dataset_name))
    print("[train] num data:", train_mat.shape[0])
    print("[test]  num data:", test_mat.shape[0])
    num_user = train_mat[:, 0].max()
    num_item = train_mat[:, 1].max()
    x_train, y_train, x_test, y_test = train_mat[:, :-1], train_mat[:, -1], \
                                           test_mat[:, :-1], test_mat[:, -1]
print("# user: {}, # item: {}".format(num_user, num_item))


===>Load from yahoo data set<===
[train] num data: 311704
[test]  num data: 54000
# user: 15400, # item: 1000


In [10]:
print(y_train.shape, x_train[:,0].shape, x_train[:,1].shape)
print(min(x_train[:,0]),min(x_train[:,1]))

(311704,) (311704,) (311704,)
1 1


In [7]:
from utils import *
x_train, y_train = shuffle(x_train, y_train)
y_train = binarize(y_train)
y_test = binarize(y_test)
from scipy import sparse as sps
y_train_sparse = sps.csr_matrix((y_train, (x_train[:, 0], x_train[:, 1])), shape=(num_user,                                     num_item), dtype=np.float32) # (290, 300)
y_train_sparse = y_train_sparse.toarray().reshape(num_user * num_item) # (87000, 1)
O_train_sparse = sps.csr_matrix((np.ones(len(x_train[:, 0])), (x_train[:, 0], x_train[:, 1])),                                  shape=(num_user, num_item), dtype=np.float32)
O_train_sparse = O_train_sparse.toarray().reshape(num_user * num_item)

ValueError: row index exceeds matrix dimensions

In [5]:
"MF"
from models import MF, MF_IPS, MF_SNIPS, MF_DR, MF_DR_JL
embedding_k = 8
mf = MF(num_user, num_item, embedding_k = embedding_k)
mf.cuda()
hyper = {"num_epoch":1000, "batch_size":128, "lr":0.01, "lamb":1e-4, "tol":1e-5, "verbose":False, "G": 1}
mf.fit(x_train, y_train, hyper, y_ips=y_train_sparse)

[MF] epoch:0, xent:nan
[MF] epoch:10, xent:nan
[MF] epoch:20, xent:nan
[MF] epoch:30, xent:nan
[MF] epoch:40, xent:nan
[MF] epoch:50, xent:nan
[MF] epoch:60, xent:nan
[MF] epoch:70, xent:nan
[MF] epoch:80, xent:nan
[MF] epoch:90, xent:nan
[MF] epoch:100, xent:nan
[MF] epoch:110, xent:nan
[MF] epoch:120, xent:nan


KeyboardInterrupt: 

In [ ]:
sample = generate_meshgrid(num_user, num_item) # (87000, 2)
y_train_hat, emb= mf.predict(sample) # (87000, 1), (87000, 16)

In [ ]:
sigma_h = 1
W_d = np.random.normal(0, sigma_h, 2 * embedding_k) # (16,)
lbd = np.exp(np.dot(emb, W_d)) # (87000,)
D = np.random.exponential(lbd) # (87000,)

L = np.quantile(D[y_train_sparse == 1], 0.6) # (1,) get the 60% value
ts_click = np.random.uniform(0, L, num_user * num_item) # (87000,)
E = L - ts_click # (87000,)
idx = (D <= E)
print(np.sum(y_train))
y_train = y_train_sparse * idx
print(np.sum(y_train))

In [ ]:
y_train_sparse[O_train_sparse == 0] = -1
E[y_train == 1] = D[y_train == 1]

In [ ]:
train_data = np.c_[np.c_[np.c_[np.c_[np.array(sample), O_train_sparse], y_train_sparse], y_train], E]

In [ ]:
import pandas as pd
train_data = pd.DataFrame(train_data, columns = ['Uid', 'Iid', 'O', 'C', 'Y_obs', 'E'])
train_labels = np.c_[np.c_[np.c_[y_train_sparse, D], y_train], E]
print(train_labels)

In [ ]:
print(y_train[y_train==y_train_sparse].shape)

In [ ]:
print(y_train[y_train*y_train_sparse==1].shape)
print(set(y_train[y_train_sparse==-1])) # c=-1 -> yobs=0
print(set(y_train))
print(set(y_train-y_train_sparse))

In [ ]:
print(f"yobs=1: {y_train[y_train==1].shape}")
print(f"yobs=0: {y_train[y_train==0].shape}")
print(f"c=1: {y_train_sparse[y_train_sparse==1].shape}")
print(f"c=-1: {y_train_sparse[y_train_sparse==-1].shape}")
print(f"c=0: {y_train_sparse[y_train_sparse==0].shape}")

(c = -1) -> no click -> (yobs = 0) -> 没看到 -> 1 − po
(c = 0) -> no conversion -> (yobs = 0) -> 看到了不喜欢 -> po(1 − pc)
(c = 1)*(D>E) -> (yobs = 0) -> 看到了，喜欢没来得及买 -> po*pc*exp(-λ*E)
(c = 1)*(D<=E) -> (yobs = 1) -> 看到了，喜欢，买了 -> -(log(po) + log(pc) + log(λ) − λ*D)
前三行加起来取负对数

In [ ]:
y_hat_bin = [1 if x>=0.5 else 0 for x in y_train_hat]
yy = np.c_[np.array(y_train), np.array(y_hat_bin)].astype(int)
yy

In [ ]:
y_test_pred, _ = mf.predict(x_test)
y_test_pred = [1 if x>=0.5 else 0 for x in y_test_pred]
print(f"mse: {mse(y_test_pred, y_test)}")
print(f"acc: {acc(y_test_pred, y_test)}")

In [6]:
from sklearn.metrics import roc_auc_score

# Example true labels (0 and 1)
y_label = [0, 0, 1, 1, 0, 1, 0, 1, 1, 0]

# Example predicted probabilities for the positive class (1)
pred_prob = [0.1, 0.4, 0.35, 0.8, 0.4, 0.9, 0.3, 0.75, 0.65, 0.5]

# Compute the ROC AUC score
auc = roc_auc_score(y_label, pred_prob)

print(f"ROC AUC score: {auc:.2f}")

ROC AUC score: 0.88
